In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os\
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SyntaxError: invalid syntax (940212519.py, line 12)

In [3]:
import torch
import numpy as np
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Paths to JSON files
train_path = "C:\Users\User\Documents\Math Classification\Code and Dataset\train.json"
test_path = "C:\Users\User\Documents\Math Classification\Code and Dataset\test.json"

# Define label mapping
label_mapping = {
    "Computations with Matrices": 0,
    "Determinants": 1,
    "Eigenvalues and Eigenvectors": 2,
    "Linear Programming and Game Theory": 3,
    "Matrices and Gaussian Elimination": 4,
    "Orthogonality": 5,
    "Positive Definite Matrices": 6,
    "Vector Spaces": 7
}

# Load train and test JSON
def load_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict):
        possible_keys = ["data", "questions", "examples"]
        for key in possible_keys:
            if key in data and isinstance(data[key], list):
                data = data[key]
                break
        else:
            raise ValueError(f"Expected list in {json_path}, but got dictionary without a valid key.")

    if not isinstance(data, list):
        raise ValueError(f"Expected a list of dictionaries in {json_path}, but got {type(data)}.")

    texts, labels = [], []
    
    for item in data:
        if not isinstance(item, dict):
            raise ValueError(f"Invalid item in JSON: {item}. Expected a dictionary.")
        if "question_latex" not in item or "chapter" not in item:
            raise KeyError(f"Missing required keys in JSON item: {item}")

        if item["chapter"] not in label_mapping:
            raise ValueError(f"Unknown chapter '{item['chapter']}' in JSON file.")

        texts.append(item["question_latex"])
        labels.append(label_mapping[item["chapter"]])  # Map chapter to label
        
    return texts, labels

train_texts, train_labels = load_data(train_path)
test_texts, test_labels = load_data(test_path)

# Load SciBERT tokenizer and model
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_mapping), ignore_mismatched_sizes=True
)

# Tokenize datasets
def tokenize_function(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=512)

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "label": train_labels
})

test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "label": test_labels
})

# Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    conf_matrix = confusion_matrix(labels, predictions)

    print("\nConfusion Matrix:")
    print(conf_matrix)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./scibert_math_classification",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Disables WandB logging if not used
)

# Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save model & tokenizer
model.save_pretrained("./scibert_math_model")
tokenizer.save_pretrained("./scibert_math_model")

# Evaluate the model
results = trainer.evaluate()
print("\nEvaluation Metrics:")
print(results)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2177118512.py, line 9)